# Bibliotecas

In [ ]:
import dedupe

import pandas as pd
import numpy as np

from pathlib import Path
from pyhive import hive

In [ ]:
# Configs HIVE
configuracao_banco = {
    'host': '10.21.0.70',
    'port': '10500',
    'username': 'trilhasgsi',
    'password': 'UFMGtrilhas2020',
    'auth': 'CUSTOM',
    'database': 'mdm_v2'
}
# Schema=(par1, par2, score)
tabela_scores = 'mdm_v2.tmp_scores_exato_26_11_2020'


# Configs modelo
arquivo_modelo = Path("../scripts/modelo exato")
num_cores = 30

# Carregando o Modelo Treinado

In [ ]:
# Modelo dedupe
with arquivo_modelo.open(mode="rb") as fp:
    modelo_dedupe = dedupe.StaticDedupe(settings_file=fp, num_cores=num_cores)

# Pegando Dados do HIVE

In [ ]:
# Configuracao HIVE


In [ ]:
# Configuracao selecao dados
atributos = ['par_1', 'par_2', 'score']

atributos_select = ', '.join([f'{x}' for x in atributos])
query = f"""
SELECT {atributos_select}  
FROM {tabela_scores}
"""
print('QUERY:')
print(query)

In [ ]:
# Lendo dados do HIVE
read_con = hive.connect(**configuracao_banco)

dados_scores = []
with read_con.cursor() as read_cur:
    read_cur.execute(query)

    for tup in read_cur:
        tup_dict = dict(zip(atributos, tup))

        pares = (tup_dict['par_1'], tup_dict['par_2'])
        score = tup_dict['score']
#         pares = np.array((tup_dict['par_1'], tup_dict['par_2']))
#         score = np.float32(tup_dict['score'])
        dados_scores.append((pares, score))

# Formato de input para o Dedupe
dados_scores = np.array(dados_scores, dtype=[('pairs', '<i8', (2,)), ('score', '<f4')])

# Fecha conexao
read_con.close()

Formato final dos scores:

In [ ]:
dados_scores

# Clusterização

In [ ]:
grupos = modelo_dedupe.cluster(dados_scores, threshold=0)
list(grupos)